# Week1 Lab - Trax and Layers

In [8]:
import numpy as np  # regular ol' numpy

from trax import layers as tl  # core building block
from trax import shapes  # data signatures: dimensionality and type
from trax import fastmath  # uses jax, offers numpy on steroids

## Relu Layer

One of the simplest types of layers. Works like a math function.

In [9]:
relu = tl.Relu()

print("___Properties___")
print(f"name: {relu.name}")
print(f"expected inputs: {relu.n_in}")
print(f"promised outputs: {relu.n_out}")

x = np.array([-2, -1, 0, 1, 2])
print("___Inputs___")
print(f"x: {x}")

y = relu(x)
print("__Outputs__")
print(f"y: {y}")

___Properties___
name: Serial
expected inputs: 1
promised outputs: 1
___Inputs___
x: [-2 -1  0  1  2]
__Outputs__
y: [0 0 0 1 2]


## Concatenate Layer
Takes two inputs

In [10]:
concat = tl.Concatenate()
print("__Properties__")
print(f"name: {concat.name}")
print(f"expected inputs: {concat.n_in}")
print(f"promised outputs: {concat.n_out}")

x1 = np.array([-10, -20, -30])
x2 = x1 / -10
print("__Inputs__")
print(f"x1: {x1}")
print(f"x2: {x2}")

y = concat([x1, x2])
print("__Outputs__")
print(f"y: {y}")

__Properties__
name: Concatenate
expected inputs: 2
promised outputs: 1
__Inputs__
x1: [-10 -20 -30]
x2: [1. 2. 3.]
__Outputs__
y: [-10. -20. -30.   1.   2.   3.]


## Layers are Configurable

In [11]:
concat_3l = tl.Concatenate(n_items=3)
print("__Properties__")
print(f"name: {concat_3l.name}")
print(f"expected_inputs: {concat_3l.n_in}")
print(f"promised outputs: {concat_3l.n_out}")

x1 = np.array([-10, -20, -30])
x2 = x1 / 10
x3 = x2 * 0.99

print("__Inputs__")
print(f"x1: {x1}")
print(f"x2: {x2}")
print(f"x3: {x3}")

y = concat_3l([x1, x2, x3])
print("__Outputs__")
print(f"y: {y}")

__Properties__
name: Concatenate
expected_inputs: 3
promised outputs: 1
__Inputs__
x1: [-10 -20 -30]
x2: [-1. -2. -3.]
x3: [-0.99 -1.98 -2.97]
__Outputs__
y: [-10.   -20.   -30.    -1.    -2.    -3.    -0.99  -1.98  -2.97]


## Layers can have weights

Some layer types include mutable weights and biases that are used in computation and training. They need to be initialized before use.

The layer LayerNorm calculates normalized data which is scaled by weights and biases.

In [12]:
norm = tl.LayerNorm()

x = np.array([0, 1, 2, 3], dtype="float")

norm.init(shapes.signature(x)) # convert input datatype to trax ShapeDType

print(f"Normal shape: {x.shape} DataType: {type(x.shape)}")
print(f"Shapes Trax: {shapes.signature(x)} DataType: {type(shapes.signature(x))}")

print("__Properties__")
print(f"name: {norm.name}")
print(f"expected inputs: {norm.n_in}")
print(f"promised outputs: {norm.n_out}")

#weights and biases
print(f"weights: {norm.weights[0]}")
print(f"biases: {norm.weights[1]}")

# Inputs
print("-- Inputs --")
print("x :", x)

# Outputs
y = norm(x)
print("-- Outputs --")
print("y :", y)

Normal shape: (4,) DataType: <class 'tuple'>
Shapes Trax: ShapeDtype{shape:(4,), dtype:float64} DataType: <class 'trax.shapes.ShapeDtype'>
__Properties__
name: LayerNorm
expected inputs: 1
promised outputs: 1
weights: [1. 1. 1. 1.]
biases: [0. 0. 0. 0.]
-- Inputs --
x : [0. 1. 2. 3.]
-- Outputs --
y : [-1.3416404  -0.44721344  0.44721344  1.3416404 ]


## Custom Layers

In [13]:
def TimesTwo():
    layer_name = "TimesTwo"

    def func(x):
        return x * 2

    return tl.Fn(layer_name, func)

times_two = TimesTwo()

# Inspect properties
print("-- Properties --")
print("name :", times_two.name)
print("expected inputs :", times_two.n_in)
print("promised outputs :", times_two.n_out, "\n")

# Inputs
x = np.array([1, 2, 3])
print("-- Inputs --")
print("x :", x, "\n")

# Outputs
y = times_two(x)
print("-- Outputs --")
print("y :", y)

-- Properties --
name : TimesTwo
expected inputs : 1
promised outputs : 1 

-- Inputs --
x : [1 2 3] 

-- Outputs --
y : [2 4 6]


## Combinators

In [14]:
serial = tl.Serial(
    tl.LayerNorm(),
    tl.Relu(),
    times_two
)

x = np.array([-2, -1, 0, 1, 2]) #input
serial.init(shapes.signature(x)) #initialising serial instance

print("-- Serial Model --")
print(serial,"\n")
print("-- Properties --")
print("name :", serial.name)
print("sublayers :", serial.sublayers)
print("expected inputs :", serial.n_in)
print("promised outputs :", serial.n_out)
print("weights & biases:", serial.weights, "\n")

# Inputs
print("-- Inputs --")
print("x :", x, "\n")

# Outputs
y = serial(x)
print("-- Outputs --")
print("y :", y)

-- Serial Model --
Serial[
  LayerNorm
  Serial[
    Relu
  ]
  TimesTwo
] 

-- Properties --
name : Serial
sublayers : [LayerNorm, Serial[
  Relu
], TimesTwo]
expected inputs : 1
promised outputs : 1
weights & biases: ((Array([1, 1, 1, 1, 1], dtype=int32), Array([0, 0, 0, 0, 0], dtype=int32)), ((), (), ()), ()) 

-- Inputs --
x : [-2 -1  0  1  2] 

-- Outputs --
y : [0.        0.        0.        1.4142132 2.8284264]
